In [29]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("crewai.ipynb")

logger.info("logger is set up and ready to rock")

INFO:crewai.ipynb:logger is set up and ready to rock


# Crew Ai Sandbox

## What is Crew AI?

CrewAi is a framework to make it easier to work with roleplaying AI (LLM) agents to achieve a tree of thought. It is open source and its source code may be found on [GitHub](https://github.com/crewAIInc/crewAI)

## Concepts involved

* LLM (Large Language Models)
* Prompt Engineering

## Setting up

I recommend using `virtualenv`, once you have it create a new `virtualenv` on this folder, active it (by `source`ing on Unix systems the `activate` script) and restore dependencies by running `pip install -r requirements.txt`.

You'll also want to have `ollama` running locally in order to pull and use local LLMs. In order to save money this sandbox uses local environments instead of expensive OpenAI calls. Ollama can be found at [their official website](https://ollama.com/download).

To make it easier to interact with ollama without doing manual steps in the terminal or gui we'll be using `ollama-python` to set things up using the notebook itself.

### Problems with Python 3.13

If you are using Python 3.13 you might run into issues (as of 2024-12-14) when installing CrewAi thru pip.

In order to get my environment to work I had to:

1. Set up my machine to run `rust` by using the script from [RustUp.rs](https://rustup.rs)
2. Add rust's cargo tools to my path by sourcing it on `.zshrc`
3. Exporting a specific variable in order to toggle compatibility between 3.12 and 3.13 when compiling: `export PYO3_USE_ABI3_FORWARD_COMPATIBILITY=1`

## Experimenting

### Installing the required libraries

As noted previously it's important to have a local copy of the requirements denoted in `requirements.txt`, if running only the notebook this setup phase may be achieved by running the cell below which will install the required dependencies.

I recommend still using at the very least a `venv` for this notebook not to mess up any system-wide dependencies you might have in your own environment.

In [32]:
# note: if you have already restored dependencies by looking at the requirements.txt file, you can skip this cell
# %pip runs pip directly from the notebook
%pip install ollama
%pip install crewai

261727.83s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


261733.80s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


### Ensuring that ollama is available in the environment

Before committing to a solution I would like to ensure that it is possible to have things set in a way that they are repeatable so, I'll begin this experiment by looking at the `ollama` library and if (using it) we can tell whenever ollama is or isn't running.

From looking at their docs seems like the simplest way to ensure ollama is available is to try and `list` models as, even if there are no models available, the result is non-error. That way if there is an exception it means that somehow `ollama` either isn't available or isn't set up properly.

In [30]:
logger.info("ensuring that ollama is up and running")
from ollama import list

try:
    list()
    logger.info("ollama is up and running")
except:
    logger.error("ollama is not running, exiting")
    exit(1)


INFO:crewai.ipynb:ensuring that ollama is up and running
INFO:httpx:HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
INFO:crewai.ipynb:ollama is up and running


Once we know it's available we can go ahead and pull from one model we know it's available. As of the time of this experiment the model I'm most aware of is `llama3` so I'll use it as the desired one.

An extensive list of available models may be found on [ollama's website](https://ollama.com/library).

Once a model is pulled and available locally we can prompt it, without creating a chat, by using the `generate` function and passing in a prompt we`d like to ask it.

In [33]:
from ollama import pull, show, generate
DESIRED_MODEL = "llama3"

# attempt to show the model, if it doesn't exist it means we need to pull it
try:
    show(DESIRED_MODEL)
    logger.info(f"model {DESIRED_MODEL} found locally, no need to pull")
except:
    logger.info(f"model {DESIRED_MODEL} not found, pulling it")
    pull(DESIRED_MODEL)
    logger.info(f"model {DESIRED_MODEL} pulled successfully")

logger.info("asking the model why the sky is blue")
response = generate(DESIRED_MODEL, "why is the sky blue, in up to 256 characters")
logger.info(f"model's response: {response.response}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/show "HTTP/1.1 200 OK"
INFO:crewai.ipynb:model llama3 found locally, no need to pull
INFO:crewai.ipynb:asking the model why the sky is blue
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:crewai.ipynb:model's response: The sky appears blue because of a phenomenon called Rayleigh scattering, where shorter (blue) wavelengths are scattered more than longer (red) wavelengths by tiny molecules of gases like nitrogen and oxygen in the atmosphere. This scattering gives the sky its blue color!


model='llama3' created_at='2024-12-16T03:17:54.31684Z' done=True done_reason='stop' total_duration=4517865375 load_duration=544011666 prompt_eval_count=22 prompt_eval_duration=2073000000 eval_count=51 eval_duration=1899000000 response='The sky appears blue because of a phenomenon called Rayleigh scattering, where shorter (blue) wavelengths are scattered more than longer (red) wavelengths by tiny molecules of gases like nitrogen and oxygen in the atmosphere. This scattering gives the sky its blue color!' context=[128006, 882, 128007, 271, 35734, 374, 279, 13180, 6437, 11, 304, 709, 311, 220, 4146, 5885, 128009, 128006, 78191, 128007, 271, 791, 13180, 8111, 6437, 1606, 315, 264, 25885, 2663, 13558, 64069, 72916, 11, 1405, 24210, 320, 12481, 8, 93959, 527, 38067, 810, 1109, 5129, 320, 1171, 8, 93959, 555, 13987, 35715, 315, 45612, 1093, 47503, 323, 24463, 304, 279, 16975, 13, 1115, 72916, 6835, 279, 13180, 1202, 6437, 1933, 0]


### Crew AI - without Tools

By this point we have a working LLM locally that is able to be manually prompted and interacted with in a simple manner.

This is the basis in which CrewAI expands upon by providing a framework which can simulate a 'crew' composed by many agents (in this case AI ones) that interact with one another in order to achieve a task. They may also be handed out tools but we'll explore this later!

From [CrewAI's docs](https://docs.crewai.com/introduction#how-crewai-works) this is a process in which we:

1. Create agents - the team members of your crew - that will be created upon a specific LLM Model with a specific role and a backstory
2. Create tasks - the work to be done - that will be assigned to specific agents in a crew
3. Create a crew - the team itself - by bringing in the agents, their tasks and a process they should follow to achieve an outcome

Linking this up to real world examples:

| CrewAI Concepts | Real life equivalent | Docs |
| --------------- | -------------------- | ---- |
| Agent | A software engineer in an Agile Squad | https://docs.crewai.com/concepts/agents |
| Task | A JIRA ticket an SE needs to work on | https://docs.crewai.com/concepts/tasks |
| Process | The development workflow they should follow | https://docs.crewai.com/concepts/processes |
| Crew | An agile squad composed of many members (Agents) of different roles (SE, SDET, UX) | https://docs.crewai.com/concepts/crews |

#### First problem: Create a landing page for a static website

As a software engineer I know how tempting it is to over-engineer something simple (such as this being a notebook rather than an one-off script lol) so I'll simulate a problem that would test if a LLM falls into such a trap if he/she is part of an Agile squad that happens to have a SE, an UX and a Business Owner.

My expected outcome is that by orchestrating those agents we may reach to:

1. requirements for the software engineer to work on, provided by a business owner
2. a simple solution, provided by the engineer, based on the owner's requirements
3. a high-level list of stories the engineer could work on based on, created by the business owner and the engineer together

Based on CrewAI docs I'll begin by creating the agents:

##### Business Owner: Karoline

Role: `Business Owner of a Candy Shop`

Backstory: 
```
Your name is Karoline and you are a small business owner of a candy shop called Karoline's Kandy, in Curitiba - Brazil, that sells candies such as Brigadeiros, Brownie and Cakes as well as gourmet coffee. You aren't keen on technology but knows that having your business show up on Social Medias and having its own link helps people to find it. You have a limited budget for this project because you just rented a venue close to a huge avenue where you expect both locals and tourists to pass by your store. Right now you are very reluctant to pursue virtual sales due to the complicated logistics required to support it.
```

Goal: `have a simple landing page for my business and an idea of how much, and long, it would take to build and maintain`

##### Software Engineer: Jakan

Role: `Consultant Software Engineer`

Backstory:
```
Your name is Jakan and are a software engineer working solo, born and raised in São Paulo - Brazil, you take on freelance opportunities by connecting with people over LinkedIn and word-of-mouth. You have worked on the IT field for over 10 years now, has deployed many systems by now and is expert on backend and frontend development. You are also able to quickly research non-technical information in order to make informed decisions and to support your customers on things that aren't related to code. You take a pragmatic and DevOps approach to problems, creating code that is refactorable but isn't 'perfect' on the first pass. Your main technologies are .NET 6 using the C# language and Vue with Typescript. Your preferred cloud provider is Azure.
```

Goal: `solve problems by using technology in a way that the results are reliable yet fit within a business' budget limits`

##### Task: Define the identify of your Business 

Must be done solo.

Description:
```
Define the branding of your business, taking into account your favorite colors (Red and White), your target audience (middle class and tourists) and a limited budget. Write a report defining what colors you'd like used when referring to your brand, its logo, social media you'd like to be on, digital experiences you'd like to provide your customers and the tone you'd like to be used in your brand's communication. Be conscious of your limited budget, keep it simple.
```

##### Task: Create a system design for Karoline's landing page

Must be done together.

Description:
```
Work out with other stakeholders to define a system design that meets their needs and fits their budgets. Take into account all the steps required to deploy a system to production and making it accessible in a secure way on customer-grade browsers. Provide a report on the high-level system design you would create to solve the problem at hand, focus on artifacts's names, their purpose and interactions in a high level.
```

##### Task: Define the terraform files and resources you'll create as part of 

WIP

##### Task: Agree upon a list of stories that should be part of a Minimum Viable Product

Must be done together.

Description:
```
Given the business needs and a system design work together to highlight a simple minimum viable product that could be delivered in less than a month to quickly provide value to stakeholders. Provide a report listing out User Stories (and its Sub-Tasks, if complex) and Tasks (non-code work) that need to be done in order to achieve that MVP state.
```

In [40]:
from crewai import Agent, Task, Process, Crew, LLM

# creating an Ollama based agent
ollamaAgent = LLM(model=f"ollama/{DESIRED_MODEL}")

logger.info("defining agents")

businessOwner = Agent(
    role="business owner of Karoline's Kandy",
    goal="have a simple landing page for my business and an idea of how much, and long, it would take to build and maintain",
    backstory="""
    Your name is Karoline and you are a small business owner of a candy shop called Karoline's Kandy, in Curitiba - Brazil, that sells candies such as Brigadeiros, Brownie and Cakes as well as gourmet coffee. 
    You aren't keen on technology but knows that having your business show up on Social Medias and having its own link helps people to find it. 
    You have a limited budget for this project because you just rented a venue close to a huge avenue where you expect both locals and tourists to pass by your store. 
    Right now you are very reluctant to pursue virtual sales due to the complicated logistics required to support it""",
    verbose=True,
    allow_delegation=False,
    llm=ollamaAgent
)

softwareEngineer = Agent(
    role="consulting software engineer",
    goal="solve problems by using technology in a way that the results are reliable yet fit within a business' budget limits",
    backstory="""
    Your name is Jakan and are a software engineer working solo, born and raised in São Paulo - Brazil, you take on freelance opportunities by connecting with people over LinkedIn and word-of-mouth. 
    You have worked on the IT field for over 10 years now, has deployed many systems by now and is expert on backend and frontend development. 
    You are also able to quickly research non-technical information in order to make informed decisions and to support your customers on things that aren't related to code. 
    You take a pragmatic and DevOps approach to problems, creating code that is refactorable but isn't 'perfect' on the first pass. 
    Your main technologies are .NET 6 using the C# language and Vue with Typescript. 
    Your preferred cloud provider is Azure.""",
    verbose=True,
    allow_delegation=False,
    llm=ollamaAgent
)

logger.info("defining tasks")
defineIdentity = Task(
    name="Define identity",
    description="""
    Define the branding of your business, taking into account your favorite colors (Red and White), your target audience (middle class and tourists) and a limited budget. 
    Be conscious of your limited budget, keep it simple.
    """,
    expected_output="Write a report defining what colors you'd like used when referring to your brand, its logo, social media you'd like to be on, digital experiences you'd like to provide your customers and the tone you'd like to be used in your brand's communication.",
    agent=businessOwner
)

defineSystemDesign = Task(
    name="Define system design",
    description="""
    Work out with other stakeholders to define a system design that meets their needs and fits their budgets.
    When in doubt if you should make something simple or complex consult with your stakeholder and their budget and needs.
    Take into account all the steps required to deploy a system to production and making it accessible in a secure way on customer-grade browsers. 
    """,
    expected_output="Provide a report, in a markdown format, of the high-level system design you would create to solve the problem at hand, focus on artifacts's names, their purpose and interactions in a high level.",
    agent=softwareEngineer,
    context=[defineIdentity]
)

defineMvp = Task(
    name="Define MVP",
    description="""
    Given the business needs and a system design work together to highlight a simple minimum viable product that could be delivered in less than a month to quickly provide value to stakeholders.
    """,
    agent=softwareEngineer,
    expected_output="Provide a report, in a markdown format, listing out User Stories (and its Sub-Tasks, if complex) and Tasks (non-code work) that need to be done in order to achieve that MVP state.",
    context=[defineSystemDesign, defineIdentity]
)

logger.info("defining the crew")
dynamicDuo = Crew(
    agents=[businessOwner, softwareEngineer],
    tasks=[defineIdentity, defineSystemDesign, defineMvp],
    verbose=True,
    process=Process.sequential
)

logger.info("kicking off the crew")
result = dynamicDuo.kickoff()

logger.info("crew has finished")
logger.info(f"result: {result}")
print(result)

INFO:crewai.ipynb:defining agents
INFO:crewai.ipynb:defining tasks
INFO:crewai.ipynb:defining the crew
INFO:crewai.ipynb:kicking off the crew
00:34:36 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= llama3; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3; provider = ollama


# Agent: business owner of Karoline's Kandy
## Task: 
    Define the branding of your business, taking into account your favorite colors (Red and White), your target audience (middle class and tourists) and a limited budget. 
    Be conscious of your limited budget, keep it simple.
    


00:34:57 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
00:34:58 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= llama3; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3; provider = ollama




# Agent: business owner of Karoline's Kandy
## Final Answer: 
**

Branding Report for Karoline's Kandy

**Color Palette:**
To reflect my favorite colors, Red and White, I propose the following color palette for our brand:

* Primary Color: Warm Red (#FFC080)
* Secondary Color: Pure White (#FFFFFF)

These colors will be used consistently across all marketing materials, packaging, and store decor to create a recognizable and memorable brand identity.

**Logo:**
Our logo will feature a stylized, hand-drawn design that combines the warmth of red with the simplicity of white. The logo will include:

* A smiling candy icon (representing our sweet treats)
* The words "Karoline's Kandy" in bold, cursive font
* A subtle pattern of small candies or confetti surrounding the text

The logo will be used on our website, social media profiles, and marketing materials.

**Social Media Presence:**
To reach our target audience of middle-class locals and tourists, we will maintain a presence on the fol

00:35:31 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
00:35:31 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= llama3; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"




# Agent: consulting software engineer
## Final Answer: 
**System Design Report for Karoline's Kandy**

**Overview**
The goal is to create a comprehensive system that integrates with Karoline's Kandy's branding elements and provides a seamless experience for customers, while fitting within the allocated budget. This report outlines the high-level design of the system, focusing on artifacts' names, their purpose, and interactions.

### 1. System Architecture

* **Frontend**: Vue.js with TypeScript will be used to build the online store, email newsletter, and social media integrations.
* **Backend**: .NET 6 with C# will power the online store's e-commerce functionality, integrating with Azure for cloud services and scalable infrastructure.
* **Database**: A Microsoft SQL Server database on Azure will manage product information, orders, and customer data.

### 2. System Components

* **Online Store**:
	+ Catalog: Display products with images, descriptions, and prices.
	+ Cart: Allow cust

00:35:54 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:crewai.ipynb:crew has finished
INFO:crewai.ipynb:result: **MVP System Development Report for Karoline's Kandy**

### 1. User Stories

* As an online shopper, I want to browse products with images, descriptions, and prices to add/remove items from my cart.
	+ Sub-Tasks:
		- Develop the product catalog
		- Implement cart functionality (add/remove items, view subtotal/total)
* As a customer, I want to subscribe/unsubscribe to receive regular newsletters with exclusive promotions, new product announcements, and special offers.
	+ Sub-Tasks:
		- Set up email newsletter system
		- Integrate subscription management feature with online store
* As a social media user, I want to engage



# Agent: consulting software engineer
## Final Answer: 
**MVP System Development Report for Karoline's Kandy**

### 1. User Stories

* As an online shopper, I want to browse products with images, descriptions, and prices to add/remove items from my cart.
	+ Sub-Tasks:
		- Develop the product catalog
		- Implement cart functionality (add/remove items, view subtotal/total)
* As a customer, I want to subscribe/unsubscribe to receive regular newsletters with exclusive promotions, new product announcements, and special offers.
	+ Sub-Tasks:
		- Set up email newsletter system
		- Integrate subscription management feature with online store
* As a social media user, I want to engage with Karoline's Kandy through Facebook and Instagram API integrations for posting updates, sharing products, and interacting with customers.
	+ Sub-Tasks:
		- Implement Facebook and Instagram API integrations
		- Develop content delivery system for social media platforms

### 2. Tasks (non-code work)

* Research 

INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
